In [240]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from urllib import request
import numpy as np
import pandas as pd
import csv
from datetime import date
import sys
from math import ceil

In [ ]:
#https://napi.wildberries.ru/api/catalog/obuv/zhenskaya/baletki-i-cheshki?sort=popular&page=1
#zhenshchinam/odezhda/bluzki-i-rubashki
#https://www.wildberries.ru/catalog/

In [179]:
category_list = ['zhenshchinam',
'obuv',
'detyam',
'muzhchinam',
'dom-i-dacha',
'novyy-god',
'krasota',
'aksessuary',
'aksessuary/avtotovary',
'igrushki',
'dom/mebel',
'aksessuary/tovary-dlya-vzroslyh',
'pitanie',
'tsvety',
'bytovaya-tehnika',
'tovary-dlya-zhivotnyh',
'sport',
'transportnye-sredstva',
'knigi',
'yuvelirnye-ukrasheniya',
'dom-i-dacha/instrumenty',
'dachniy-sezon',
'dom-i-dacha/zdorove',
'sdelano-v-rossii',
'kulturnyy-kod']

In [181]:
# собираем все категории
all_categories = []
for i in category_list:
    r = requests.get(f"https://napi.wildberries.ru/api/catalog/{i}")
    soup = BeautifulSoup(r.text, 'html.parser')
    d = soup.get_text()
    res = json.loads(d)
    for i in res['data']['currentMenu']:
        #убираем диджитал продукты т.к. они находятся на отдельной ВБ странице
        if i['url'].find('digital')==-1:
            all_categories.append(i['url'][i['url'].find('catalog')+8:])
# есть категории, которые не собираются из napi - их прописываем отдельно
all_categories += ['knigi-i-diski/kantstovary/anatomicheskie-modeli',
'knigi-i-kantstovary/kantstovary/bumazhnaya-produktsiya',
'knigi-i-kantstovary/kantstovary/karty-i-globusy',
'knigi-i-kantstovary/kantstovary/ofisnye-prinadlezhnosti',
'knigi-i-kantstovary/kantstovary/pismennye-prinadlezhnosti',
'knigi-i-kantstovary/kantstovary/risovanie-i-lepka',
'knigi-i-kantstovary/kantstovary/risovanie-i-lepka',
'knigi-i-diski/kantstovary/schetniy-material',
'kantstovary/torgovye-prinadlezhnosti',
'knigi-i-kantstovary/kantstovary/chertezhnye-prinadlezhnosti',]

In [182]:
# собираем все коды продуктов из первых N страниц отсортированныз по популярности
N = 5
all_products = {}
counter=0
try:
    for i in all_categories:
        counter+=1
        for p in range (1,N):
            r = requests.get(f"https://napi.wildberries.ru/api/catalog/{i}?sort=popular&page={p}")  
            soup = BeautifulSoup(r.text, 'html.parser')
            d = soup.get_text()
            res = json.loads(d)
            categorie_temp = None
            categorie_temp = res.get('data',{}).get('breadCrumbs')
            if categorie_temp != None:
                categorie = []
                for C in range(len(categorie_temp)):
                    categorie.append(categorie_temp[C].get('name'))
            temp = res.get('data',{}).get('products')
            if temp != None:
                for m in temp:
                    all_products[m]=categorie
        print(f'{round(counter/len(all_categories)*100,2)}%', end = '\r')
except:
    print(i)

In [272]:
max_cat_list = []
for i in all_products.keys():
    max_cat_list.append(len(all_products[i]))
max_cat=max(max_cat_list)

#Количество ячеек для выгрузки
bucket_count = 10
#Выбор первой ячейки в случае продолжения ранее не завершенной выгрузки
START_BUCKET = 1

bucket_part = ceil(len(all_products.keys())/bucket_count)



for bucket in range(START_BUCKET,bucket_count+1):
    counter = 0
    globals()[f"full_info_bucket_{bucket}"] = {}
    bucket_floor = 0+bucket_part*(bucket-1)
    bucket_ceil = bucket_part*bucket
    full_counter = bucket_part*(bucket-1)
    if bucket_ceil > len(all_products.keys()):
        bucket_ceil = len(all_products.keys())
    for i in list(list(all_products.keys())[bucket_floor:bucket_ceil]):
        counter += 1
        full_counter += 1
        r = requests.get(f"https://card.wb.ru/cards/v2/detail?appType=1&curr=rub&dest=-1255987&spp=30&hide_dtype=10&ab_testing=false&nm={i}")
        r = r.text
        for repl in ['&#34;', '&quot','&#13;']:
            r = r.replace(repl,'')
        soup = BeautifulSoup(r, 'html.parser')
        d = soup.get_text()
        res = json.loads(d)
        globals()[f"full_info_bucket_{bucket}"].setdefault('ID', []).append(i)
        feedbek=''
        info=-1
        print(f"Full: {round(full_counter/len(all_products)*100,2)}%. Bucket {bucket}: {round(counter/bucket_ceil*100,2)}%. Now loading {i}                                                             ", end = '\r')
        for J in ['sizes', 'brand', 'name', 'reviewRating', 'feedbacks', 'viewFlags', 'root']:
            temp = res.get('data',{}).get('products')
            if temp!= None and len(temp)>0:
                if J == 'sizes':
                    sizes = res['data']['products'][0].get(J)
                    if sizes != None:
                        for I in range(0,len(sizes)):
                            price = sizes[I].get('price')
                            if price != None and info<0:
                                info=price['total']/100
                                globals()[f"full_info_bucket_{bucket}"].setdefault('price', []).append(info)
                        if info<0:
                            globals()[f"full_info_bucket_{bucket}"].setdefault('price', []).append(np.nan)
                    elif sizes == None and info<0:
                        globals()[f"full_info_bucket_{bucket}"].setdefault('price', []).append(np.nan)
                elif J == 'root':
                    root = None
                    root = res['data']['products'][0].get(J)
                    if root != None:
                        feedbek = root
                else:
                    globals()[f"full_info_bucket_{bucket}"].setdefault(J, []).append(res['data']['products'][0].get(J))
            else:
                if J == 'sizes':
                    globals()[f"full_info_bucket_{bucket}"].setdefault('price', []).append(np.nan)
                elif J == 'root':
                    globals()[f"full_info_bucket_{bucket}"].setdefault('sales_from', []).append(np.nan)
                else:
                    globals()[f"full_info_bucket_{bucket}"].setdefault(J, []).append(np.nan)
        if feedbek != '':
            info = pd.to_datetime(date.today())
            for FFB in range(1,3):
                r2 = requests.get(f"https://feedbacks{FFB}.wb.ru/feedbacks/v2/{feedbek}")
                r2 = r2.text
                for repl in ['&#34;', '&quot', '&#13;']:
                    r2 = r2.replace(repl,'')
                soup2 = BeautifulSoup(r2, 'html.parser')
                d2 = soup2.get_text()
                res2 = json.loads(d2)
                if res2['feedbackCountWithText']>0:
                    info2 = pd.to_datetime(res2['feedbacks'][-1]['createdDate'][:10])
                    if info2 < info:
                        info = info2
            globals()[f"full_info_bucket_{bucket}"].setdefault('sales_from', []).append(info)
        categories = all_products[i]
        number_one = len(categories)
        for FB in range(number_one):
            globals()[f"full_info_bucket_{bucket}"].setdefault(f'Category_{FB+1}', []).append(categories[FB])
        for FB in range(number_one,max_cat):
            globals()[f"full_info_bucket_{bucket}"].setdefault(f'Category_{FB+1}', []).append(np.nan)
        globals()[f"full_info_bucket_{bucket}"].setdefault('link', []).append(f'https://www.wildberries.ru/catalog/{i}/detail.aspx') 
        for TRY in globals()[f"full_info_bucket_{bucket}"].values():
            if len(TRY)/counter != 1:
                print(f'{i} is create error!!')
                print(f'{full_info}')
                sys.exit()
    print(f"full_info_bucket_{bucket} created!                                                                                      ")

full_info_bucket_10 created!                                                                                      


In [348]:
#Объединяем бакеты в DataFrame
all_summary = pd.DataFrame()
for i in range(1,bucket_count+1):
    summary = pd.DataFrame.from_dict(globals()[f"full_info_bucket_{i}"])
    all_summary = pd.concat([all_summary, summary])

In [386]:
#Первая дата коментария - начало продаж
all_summary['sales_from'] = pd.to_datetime(all_summary['sales_from'])
#Количество дней продаж
all_summary['daySales'] = pd.to_datetime(date.today()) - all_summary['sales_from']
all_summary['daySales']=pd.to_timedelta(all_summary['daySales']).dt.days+1
#Себестоимость продажи товара
all_summary['selfCost'] = all_summary['price']*0.85
#Оборачиваемость в день
all_summary['retentionD'] = all_summary['feedbacks']/all_summary['daySales']
#Оборачиваемость в месяц
all_summary['retentionM'] = round(all_summary['retentionD']*30)
#Себестоимость в месяц
all_summary['MCosts'] = round(all_summary['retentionM'] * all_summary['selfCost'])
#Доход в месяц
all_summary['MRevenue'] = all_summary['retentionM']*all_summary['price']*0.15
#Конверсия просмотров в продажи
all_summary['Convertation'] = all_summary['viewFlags']/all_summary['feedbacks']
all_summary[all_summary['Convertation']==np.inf]=np.nan
#Схраняем в csv
all_summary.to_csv('mycsvfile.csv', sep=',', index=False, encoding='utf-16')

In [448]:
new = all_summary[all_summary['Convertation']!=np.nan]
new = new[new['feedbacks']>=1000]
new = new[new['daySales']>=30]
new.sort_values('retentionM', ascending = False)['name'].unique()[:100]

array(['Носки ECO 401 высокие из хлопка набор 10 пар',
       'Носки ECO 401 высокие длинные из хлопка набор 10 пар',
       'Влажные детские салфетки ДПантенол Зайка 12х120шт с клапаном',
       'Влажные детские салфетки ДПантенол Зайка 6х120 шт с клапаном',
       'Футболка черная однотонная',
       'Эликсир спрей 12 в 1 многофункциональный Эксперт-уход 200 мл',
       'Ватные палочки бамбук и хлопок',
       'Подгузники для новорожденных Elite Soft 2 р 4-6 кг 100 шт',
       'Подгузники для новорожденных Elite Soft 2 р 4-6 кг 50 шт',
       'Подгузники для новорожденных Elite Soft 2 р 4-6 кг 20шт',
       'Подгузники для новорожденных Elite Soft 1 р 3-5 кг 100 шт',
       'Подгузники для новорожденных Elite Soft 1 р 3-5 кг 50 шт',
       'Подгузники для новорожденных Elite Soft 1 р 3-5 кг 20шт',
       'Подгузники для новорожденных Elite Soft 2 р 4-6 кг 164 шт',
       'Подгузники для новорожденных Elite Soft 0 р 0-3 кг 25 шт',
       'Бриджи домашние', 'Бриджи трикотажные домашние